# Motif specification
Each motif string contains a pdb id, specification of motif amino acid chain and residue indexes, and specification of which of these residues may be redesigned.

# Example motifs
* "7F7P,B32-46;A7-21,B32-46;A7-21" --> two segments of motif. Residues 32 through 46 on chain B and residues 7 through 21 on chain A.  All positions may be redesigned.

* "4xoj,A55;A99;A190-192,A191" --> three segments of motif.  All on chain A.  Residues 55, 99, 190, 191, and 192.  Only residue 191 may be redesigned.

# Outputs
The output of the processing is a pdb file with the coordinates of motif residues.
* Each segment of the motif is labeled as its own chain, in order A, B, C, etc.
* For that may be redesigned, all atoms other than N, CA, C and O are removed and the residue type is set to UNK.
* The header of the motif includes a contig specifying how the motif is placed in the reference scaffold.
  * Example:  consider "4xoj,A55;A99;A190-192,A191" the header contig should be "38;A;43;B;90;C;46"
  * 4xoj has 223 resolved residues (indexed as 16 through 238), the 38 corresponds to the 38 residues 16-54 before residue 55 (segment A), the 43 corresponds to the residues between residue 55 and 99 and so on. The final 46 indicates that the reference structure terminates with 46 additional residues that are not part of the motif.

In [1]:
import os
import shutil
import urllib.request
import Bio
import numpy as np
from Bio.PDB import PDBParser, Select
motif_chain_id_order = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [2]:
def parse_contig_string(contig_string):
    contig_segments = []
    for motif_segment in contig_string.split(";"):
        segment_dict ={"chain":motif_segment[0]}
        if "-" in motif_segment:
            segment_dict["start"], segment_dict["end"] = motif_segment[1:].split("-")

        else:
            segment_dict["start"] = segment_dict["end"] = motif_segment[1:]
        contig_segments.append(segment_dict)
    return contig_segments

def check_if_in_segment(redesign_segments, chain, residue):
    for segment in redesign_segments:
        if segment["chain"] == chain:
            if int(segment["start"]) <= residue <= int(segment["end"]):
                return True
    return False

class AltLocSelect(Select):
    def accept_atom(self, atom):
        # Accept atoms with altloc 'A' or no altloc, and if occupancy is 1.0
        return atom.altloc in ('A', ' ') and atom.occupancy == 1.0

def remove_alt_conformations(structure):
    # Remove atoms with alternate location identifiers other than 'A'
    for model in structure:
        for chain in model:
            for residue in chain:
                # Create a list of atoms to remove
                atoms_to_remove = []
                for atom in residue:
                    #if atom.altloc not in ('A', ' '):
                    if atom.get_full_id()[4][1] not in ('A', ' '):
                        atoms_to_remove.append(atom)
                    # Set altloc to ' ' for all atoms, and occupancy to 1.0
                    atom.set_altloc(' ')
                    atom.set_occupancy(1.0)
                if atoms_to_remove:
                    for atom in atoms_to_remove:
                        # Before removing, check that the atom is redundant and can be removed.
                        # Specifically, check that the atom name (e.g. CA) is the same as another atom in the residue
                        atom_name = atom.get_name()
                        if len([a for a in residue if a.get_name() == atom_name]) > 1:
                            residue.detach_child(atom.id)
    return structure

In [3]:
# Fetch the PDB file from RCSB and parse it with BioPython
def load_pdb(pdb_id, reference_pdb_dir="./reference_pdbs/"):
    pdb_fn = f"{reference_pdb_dir}{pdb_id}.pdb" # File name to save the PDB file to
    if pdb_id != "1QY3":
        urllib.request.urlretrieve(
            f'http://files.rcsb.org/download/{pdb_id}.pdb',
            f"{reference_pdb_dir}{pdb_id}.pdb")
    else:
        # If "1qy3" this is the pre-cyclized GFP, and we must used a hacked 
        # file with an A96R mutation (as described in the ESM3 paper).
        pdb_fn = "../test_cases/ESM3/1qy3_A96R.pdb"

    # Parse the PDB file and extract the motif segments.
    # Then save a new pdb file with each segment concatenated, with each segment in a separate chain.
    parser = PDBParser()
    structure = parser.get_structure(pdb_id, pdb_fn)
    return structure


def parse_motif_into_new_structure(structure, motif_segments, redesign_segments):
    # Create a new structure object to hold the motif segments
    motif_structure = Bio.PDB.Structure.Structure("motif")
    motif_structure.add(Bio.PDB.Model.Model(0))

    # Iterate through segments creating a new chain for each segment 
    for i, segment in enumerate(motif_segments):
        motif_structure[0].add(Bio.PDB.Chain.Chain(motif_chain_id_order[i]))

        # Iterate through residues in the segment and add them to the new chain.
        for residue in structure[0][segment["chain"]]:
            if int(segment["start"]) <= residue.id[1] <= int(segment["end"]):

                # Remove all hydrogen atoms.
                for atom in list(residue):
                    if atom.element == "H":
                        residue.detach_child(atom.id)

                # Check if the residue is in a redesign segment.
                # If so, remove atoms except the backbone and set type to UNK.
                if check_if_in_segment(redesign_segments, segment["chain"], residue.id[1]):
                    for atom in list(residue):
                        if atom.id not in ["N", "CA", "C", "O"]:
                            residue.detach_child(atom.id)
                    residue.resname = "UNK"
                
                # Reset residue index to start from 1.
                residue = residue.copy()
                residue.id = (" ", residue.id[1]-int(segment["start"])+1, " ")

                # Add the residue to the new chain.
                motif_structure[0][motif_chain_id_order[i]].add(residue)

    return motif_structure


# Center the motif structure on the origin.
# Because of some BioPython weirdness, we have to do this manually,
# by reading the saved pdb file and calculating the center of mass,
# then translating the structure to the origin, and saving it again.
def center_pdb(motif_fn):
    # Load the motif structure from the saved pdb file.
    parser = PDBParser()
    motif_structure = parser.get_structure("motif", motif_fn)


    ### Translate the motif structure to the origin.
    # Get the center of mass of the motif structure.
    com = np.array([0., 0., 0.])
    count = 0
    for atom in motif_structure.get_atoms():
        atom_vec = atom.get_vector().get_array()
        com += atom_vec
        count += 1
    com = com/count

    # Translate the motif structure to the origin.
    for atom in motif_structure.get_atoms():
        atom.set_coord(atom.get_coord() - com)

    # Save the centered motif structure, overwriting the original file.
    io = Bio.PDB.PDBIO()
    io.set_structure(motif_structure)
    io.save(motif_fn)

In [4]:
def build_contig_string(structure, motif_segments):
    # Write a contig string describing the placement of the motif in the original structure, restricting to resolved residues.

    # First find the first resolved residue in the structure.
    first_residue = None
    motif_chain = motif_segments[0]["chain"]
    for residue in structure[0][motif_chain]:
        if residue.id[1] != " ":
            first_residue = residue.id[1]
            break

    # Iterate through the motif segments and compute the number of residues between them.
    contig_string = ""
    for i, segment in enumerate(motif_segments):
        chain = motif_chain_id_order[i]
        if i == 0:
            contig_string += f"{int(segment['start']) - first_residue};{chain};"
        else:
            contig_string += f"{int(segment['start']) - int(motif_segments[i-1]['end']) - 1};{chain};"
        if i == len(motif_segments) - 1:
            break

    # get index of last residue in the structure, excluding heteroatoms
    chain = segment["chain"]
    last_AA_idx = 0
    for residue in structure[0][chain]:
        if residue.id[0] == " ": # check that it is not a heteroatom
            last_AA_idx = residue.id[1]

    contig_string += f"{last_AA_idx - int(segment['end'])}"
    return contig_string


def save_motif_pdb(motif_string, motif_pdb_dir="./motif_pdbs/", reference_pdb_dir="./reference_pdbs/", idx=None):
    pdb_id, motif_residues, redesign_residues, total_length, _ = motif_string.split(",")

    # Load structure and parse motif into new structure
    structure = load_pdb(pdb_id, reference_pdb_dir=reference_pdb_dir)

    # Some pdbs have alternate conformations, which we remove.
    structure = remove_alt_conformations(structure)

    motif_segments = parse_contig_string(motif_residues)
    redesign_segments = parse_contig_string(redesign_residues) if redesign_residues else []
    motif_structure = parse_motif_into_new_structure(structure, motif_segments, redesign_segments)

    # Save the new structure to a PDB file with a header that includes the place of the motif in the original structure.
    contig_string = build_contig_string(structure, motif_segments)
    header_string = f"REMARK 1 Reference PDB ID: {pdb_id}\n"
    header_string += f"REMARK 2 Motif Segment Placement in Reference PDB: {contig_string}\n"
    header_string += f"REMARK 3 Length for Designed Scaffolds: {total_length}\n"

    motif_fn = f"{motif_pdb_dir}{pdb_id}.pdb"
    if idx is not None:
        motif_fn = f"{motif_pdb_dir}{idx:02d}_{pdb_id}.pdb"
        print("motif_fn:", motif_fn)
    io = Bio.PDB.PDBIO()
    io.set_structure(motif_structure)
    io.save(motif_fn , AltLocSelect(), write_end=True)

    # Center the motif structure on the origin.
    center_pdb(motif_fn)
    
    # Prepend header_string to the new PDB file by copying the file and writing the header first.
    with open(motif_fn, "r") as f: file_string = f.read()
    with open(motif_fn, "w") as f: f.write(header_string + file_string)

In [5]:
motif_specs_paths = [
    #"../test_cases/rfdiffusion_benchmark/motif_specs.csv",
    #"../test_cases/orphans/motif_specs.csv",
    #"../test_cases/other_enzymes/motif_specs.csv",
    #"../test_cases/ESM3/motif_specs.csv"
    "../motif_specs.csv"
]

# Make sure the motif_pdb_dir and reference_pdb_dir directories exist.
motif_pdb_dir = "../motif_pdbs/"
if not os.path.exists(motif_pdb_dir): os.makedirs(motif_pdb_dir)

reference_pdb_dir = "../reference_pdbs/"
if not os.path.exists(reference_pdb_dir): os.makedirs(reference_pdb_dir)

# Parse the motif specs files and save the motif PDBs.
for motif_specs_path in motif_specs_paths:
    with open(motif_specs_path, "r") as f:
        f.readline() # header
        for idx, line in enumerate(f):
            print(f"{idx+1:02d}", line.strip())
            save_motif_pdb(line.strip(), motif_pdb_dir=motif_pdb_dir, reference_pdb_dir=reference_pdb_dir, idx=idx+1)

01 1BCF,A18-25;A47-54;A92-99;A123-130,A19-25;A47-50;A52-53;A92-93;A95-99;A123-126;A128-129,125,1


/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 17010.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 17055.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 17057.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 17102.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 17104.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder

motif_fn: ../motif_pdbs/01_1BCF.pdb
02 1YOV,B213-223,,75,1


/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 15357.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 15358.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 15359.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 15412.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 15474.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder

motif_fn: ../motif_pdbs/02_1YOV.pdb
03 2KL8,A1-7;A28-79,,100,1
motif_fn: ../motif_pdbs/03_2KL8.pdb
04 6E6R,A23-35,A23-35,75,1
motif_fn: ../motif_pdbs/04_6E6R.pdb
05 6E6R,A23-35,A23-35,200,1
motif_fn: ../motif_pdbs/05_6E6R.pdb
06 6EXZ,A556-570,A556-570,100,1
motif_fn: ../motif_pdbs/06_6EXZ.pdb
07 6EXZ,A556-570,A556-570,200,1
motif_fn: ../motif_pdbs/07_6EXZ.pdb
08 1ITU,A124-147,,150,2


...

/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4414.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 5286.
  warnings.warn(


motif_fn: ../motif_pdbs/26_6CPA.pdb
27 7AD5,A99-113,,125,3
motif_fn: ../motif_pdbs/27_7AD5.pdb
28 7WRK,A80-94,,125,3
motif_fn: ../motif_pdbs/28_7WRK.pdb
29 7UWL,E63-73;E101-111,E63-73;E101-103;E105-111,175,3


/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 11094.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 11108.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 11122.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 11164.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 11206.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder

motif_fn: ../motif_pdbs/29_7UWL.pdb
30 7UWL,E63-73;E101-111;E132-142;E165-174,E63-73;E101-103;E105-111;E132-142;E165-174,175,3
motif_fn: ../motif_pdbs/30_7UWL.pdb


/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 11094.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 11108.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 11122.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 11164.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 11206.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder

In [6]:
def remove_lagging_characters(filepath):
    # Ensure it's a file (not a directory or other object)
    if os.path.isfile(filepath):
        with open(filepath, 'r') as file:
            lines = file.readlines()
        
        # Process lines to modify "TER" lines
        modified_lines = []
        for line in lines:
            if line.startswith("TER"):
                modified_lines.append("TER\n")
            else:
                modified_lines.append(line)
        
        # Write the modified lines back to the file
        with open(filepath, 'w') as file:
            file.writelines(modified_lines)
def set_bfactor_to_zero(input_pdb):
    """
    Reads a PDB file, sets the B-factor value to zero for every atom, and writes the result to a new file.

    Parameters:
        input_pdb (str): Path to the input PDB file.
        output_pdb (str): Path to the output PDB file.
    """
    tmp_fn = input_pdb + ".tmp"
    with open(input_pdb, 'r') as infile, open(tmp_fn, 'w') as outfile:
        for line in infile:
            # Process only ATOM and HETATM lines
            if line.startswith(('ATOM', 'HETATM')):
                # B-factor is in columns 61-66 (1-based indexing)
                modified_line = line[:60] + f"{0.00:6.2f}" + line[66:]
                outfile.write(modified_line)
            else:
                # Write non-ATOM lines unchanged
                outfile.write(line)
    shutil.move(tmp_fn, input_pdb)

In [7]:
# Loop through all files in motif_pdb_dir and cut lagging text after "TER" lines.
for filename in os.listdir(motif_pdb_dir):
    filepath = os.path.join(motif_pdb_dir, filename)
    remove_lagging_characters(filepath)
    set_bfactor_to_zero(filepath)

In [8]:
def save_motif_pdb_genie_format(motif_string, motif_pdb_dir="./motif_pdbs/", reference_pdb_dir="./reference_pdbs/", idx=None):
    pdb_id, motif_residues, redesign_residues, total_length = motif_string.split(",")

    # Load structure and parse motif into new structure
    structure = load_pdb(pdb_id, reference_pdb_dir=reference_pdb_dir)

    # Some pdbs have alternate conformations, which we remove.
    structure = remove_alt_conformations(structure)

    motif_segments = parse_contig_string(motif_residues)
    redesign_segments = parse_contig_string(redesign_residues) if redesign_residues else []
    motif_structure = parse_motif_into_new_structure(structure, motif_segments, redesign_segments)
    print("motif segments:", motif_segments)

    # Save the new structure to a PDB file with a header that includes the place of the motif in the original structure.
    header_string = f"REMARK 999 NAME   {idx:02d}_{pdb_id}\n"
    header_string += f"REMARK 999 PDB    {pdb_id}\n"

    terminal_segment_line =    "REMARK 999 INPUT      0 200\n"
    intervening_segment_line = "REMARK 999 INPUT     15 200\n"
    header_string += terminal_segment_line
    for i, segment in enumerate(motif_segments):
        motif_chain_id = motif_chain_id_order[i]
        motif_segment_len = int(segment["end"]) - int(segment["start"]) + 1
        header_string += f"REMARK 999 INPUT  {motif_chain_id}   1{motif_segment_len:>4}\n"
        
        if i < len(motif_segments) - 1:
            header_string += intervening_segment_line
    header_string += terminal_segment_line

    # Add total length
    header_string += f"REMARK 999 MINIMUM TOTAL LENGTH      {total_length:>3}\n"
    header_string += f"REMARK 999 MAXIMUM TOTAL LENGTH      {total_length:>3}\n"
    header_string += "\n"

    motif_fn = f"{motif_pdb_dir}{pdb_id}.pdb"
    if idx is not None:
        motif_fn = f"{motif_pdb_dir}{idx:02d}_{pdb_id}.pdb"
        print("motif_fn:", motif_fn)
    io = Bio.PDB.PDBIO()
    io.set_structure(motif_structure)
    io.save(motif_fn , AltLocSelect(), write_end=True)

    # Center the motif structure on the origin.
    center_pdb(motif_fn)
    
    # Prepend header_string to the new PDB file by copying the file and writing the header first.
    with open(motif_fn, "r") as f: file_string = f.read()
    with open(motif_fn, "w") as f: f.write(header_string + file_string)

if False:
    # Parse the motif specs files and save the motif PDBs.
    motif_pdb_genie_dir = "../motif_pdbs_genie/"
    if not os.path.exists(motif_pdb_dir): os.makedirs(motif_pdb_genie_dir)
    for motif_specs_path in motif_specs_paths:
        with open(motif_specs_path, "r") as f:
            for idx, line in enumerate(f):
                print(f"{idx:02d}", line.strip())
                save_motif_pdb_genie_format(line.strip(), motif_pdb_dir=motif_pdb_genie_dir, reference_pdb_dir=reference_pdb_dir, idx=idx)

In [11]:
from Bio.PDB import Structure, Model, Chain, Residue

def create_processed_structure(structure, motif_segments, redesign_segments):
    # Create a new structure object to hold the processed structure
    processed_structure = Structure.Structure("processed_structure")
    model = Model.Model(0)
    processed_structure.add(model)

    # Create a new chain "A" for the processed structure
    new_chain = Chain.Chain("A")
    model.add(new_chain)

    # Create a set of residues that are within motif segments for easy reference
    motif_residues = set()
    for segment in motif_segments:
        motif_residues.update(
            (segment["chain"], res_id)
            for res_id in range(int(segment["start"]), int(segment["end"]) + 1)
        )

    # Copy residues from the original chain, adjusting B-factors and residue types
    for chain in structure[0]:  # Access the first model in the structure
        if not chain.id == motif_segments[0]['chain']:
            continue
        for residue in chain.get_residues():
            chain_id = chain.id
            res_id = residue.id[1]
                        
            # Skip if residue is a heteroatom
            if residue.id[0] != " ":
                continue

            for atom in list(residue):
                if atom.id not in ["N", "CA", "C", "O"]:
                    residue.detach_child(atom.id)

            # Determine if the residue is in motif segments
            if check_if_in_segment(motif_segments, chain_id, res_id):
                # Copy the residue, set B-factor to 1, and retain the original residue type
                new_residue = residue.copy()
                for atom in new_residue:
                    atom.bfactor = 1.0
            else:
                # Copy the residue, set B-factor to 0, and set type to GLY
                new_residue = residue.copy()
                for atom in new_residue:
                    atom.bfactor = 0.0
                new_residue.resname = "GLY"

            # Check if the residue is in redesign segments and set to UNK if so
            if check_if_in_segment(redesign_segments, chain_id, res_id):
                new_residue.resname = "UNK"

            # Add the modified residue to the new chain with chain "A"
            new_chain.add(new_residue)

    return processed_structure

def save_processed_pdb(motif_string, processed_pdb_dir="./processed_pdbs/", reference_pdb_dir="./reference_pdbs/", idx=None):
    pdb_id, motif_residues, redesign_residues, total_length, _ = motif_string.split(",")
    print("PDBID:", pdb_id)

    # Load structure and parse motif into new structure
    structure = load_pdb(pdb_id, reference_pdb_dir=reference_pdb_dir)

    # Some pdbs have alternate conformations, which we remove.
    structure = remove_alt_conformations(structure)

    motif_segments = parse_contig_string(motif_residues)
    redesign_segments = parse_contig_string(redesign_residues) if redesign_residues else []
    processed_structure = create_processed_structure(structure, motif_segments, redesign_segments)
    
    processed_pdb_dir_pdb = processed_pdb_dir + f"{idx:02d}_{pdb_id}/"
    if not os.path.exists(processed_pdb_dir_pdb): os.makedirs(processed_pdb_dir_pdb)
    
    if idx is not None:
        pdb_fn = f"{processed_pdb_dir_pdb}{idx:02d}_{pdb_id}.pdb"
    else:
        pdb_fn = f"{processed_pdb_dir_pdb}{pdb_id}.pdb"
        
    
    io = Bio.PDB.PDBIO()
    io.set_structure(processed_structure)
    io.save(pdb_fn , AltLocSelect(), write_end=True)
    
    center_pdb(pdb_fn)

In [14]:
motif_specs_paths = [
    "../motif_specs.csv"
]

# Make sure the motif_pdb_dir and reference_pdb_dir directories exist.
processed_pdb_dir = "../reference_pdb_baseline/"
if not os.path.exists(processed_pdb_dir): os.makedirs(processed_pdb_dir)

reference_pdb_dir = "../reference_pdbs/"
if not os.path.exists(reference_pdb_dir): os.makedirs(reference_pdb_dir)

# Parse the motif specs files and save the motif PDBs.
for motif_specs_path in motif_specs_paths:
    with open(motif_specs_path, "r") as f:
        f.readline()
        for idx, line in enumerate(f):
            print(f"{idx:02d}", line.strip())
            save_processed_pdb(line.strip(), processed_pdb_dir=processed_pdb_dir, reference_pdb_dir=reference_pdb_dir, idx=idx)

00 1BCF,A18-25;A47-54;A92-99;A123-130,A19-25;A47-50;A52-53;A92-93;A95-99;A123-126;A128-129,125,1


/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 17010.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 17055.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 17057.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 17102.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 17104.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder

01 1YOV,B213-223,,75,1


/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 15357.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 15358.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 15359.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 15412.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 15474.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder

02 2KL8,A1-7;A28-79,,100,1
03 6E6R,A23-35,A23-35,75,1
04 6E6R,A23-35,A23-35,200,1
05 6EXZ,A556-570,A556-570,100,1
06 6EXZ,A556-570,A556-570,200,1
07 1ITU,A124-147,,150,2


...

/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4414.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 5286.
  warnings.warn(


25 6CPA,A69-72;A127;A196;A248;A270,A70-71,200,3
26 7AD5,A99-113,,125,3
27 7WRK,A80-94,,125,3
28 7UWL,E63-73;E101-111,E63-73;E101-103;E105-111,175,3


/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 11094.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 11108.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 11122.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 11164.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 11206.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder

29 7UWL,E63-73;E101-111;E132-142;E165-174,E63-73;E101-103;E105-111;E132-142;E165-174,175,3


/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 11094.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 11108.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 11122.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 11164.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 11206.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder